In [1]:
# !pip install psycopg2 sqlalchemy 
# !pip install mariadb
# !pip install dask
# !pip install dask[dataframe]

In [15]:
import psycopg2
import sqlalchemy as sq
import pandas as pd
import numpy as np
import mariadb
import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path

In [16]:
import mariadb
import sys

username = "scoring"
password = "idkltb93e0eomejp"
host = 'spectral-msql-jul-24-backup-do-user-2276924-0.b.db.ondigitalocean.com'
port = 25060
database="farmlabv3_live"
# schema="farmlabv3_live"
# schema="historical"
def get_db_cursor():
    try:
        conn = mariadb.connect(
            user=username,
            password=password,
            host=host,
            port=port,
            database=database
    
        )
        return conn
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        sys.exit(1)

conn = get_db_cursor()
cur = conn.cursor()

In [ ]:
report_data = pd.read_sql("SELECT spectral_sample_id, barcode, crop_id, report_data  FROM SpectralSampleCrop ssc", con=conn)

C:\Users\tommy\AppData\Local\Temp\ipykernel_23900\2048603924.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  report_data = pd.read_sql("SELECT spectral_sample_id, barcode, crop_id, report_data  FROM SpectralSampleCrop ssc", con=conn)


In [166]:
crops = pd.read_sql("SELECT id, name FROM crop c", con=conn, index_col="name").to_dict()

C:\Users\tommy\AppData\Local\Temp\ipykernel_27780\1915557479.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  crops = pd.read_sql("SELECT id, name FROM crop c", con=conn, index_col="name").to_dict()


In [167]:
crops

{'id': {'Maize': 199,
  'sorghum': 144,
  'Soya bean (opv)': 3,
  'millet': 4,
  'coffee': 5,
  'rice': 6,
  'Maize_irrigated': 7,
  'Maize_rainfed': 8,
  'Peanuts': 9,
  'Potatoes (Irish)': 40,
  'Onions': 11,
  'Tomatoes_op': 12,
  'Tomatoes_hybrid': 13,
  'Orange Flesh Sweet Potato': 14,
  'Beans': 145,
  'Groundnuts': 16,
  'Orange Flesh Sweet Potat': 17,
  'Tobacco': 18,
  'Cabbage': 19,
  'maize': 20,
  'Maize (hybrid)': 21,
  'Carrots': 22,
  'Rhodesian blue grass': 24,
  'Napier grass': 25,
  'Maize (OPV)': 205,
  'Sunflower': 27,
  'Irish_Potatoes': 28,
  'Yam': 29,
  'Cassava': 30,
  'Yams': 31,
  'Cotton': 32,
  'Napier Grass': 33,
  'Chloris gayanas (Rhodes grass)': 36,
  'Gamba grass(Andropogon gayanas)': 35,
  'Fallow': 37,
  'pyrethrum': 38,
  'Maize (Seed)': 39,
  'Potato': 41,
  'Onion (Spring)': 42,
  'Maize Grain': 43,
  'French Bean Trash': 44,
  'Grass (Grazing)': 45,
  'Tomatoes (OP)': 46,
  'Tomatoes (Hybrid)': 47,
  'Tea': 48,
  'Snow peas': 49,
  'Peas': 50,
  

In [3]:
report_data = pd.read_csv("inputFiles/report_data.csv")

In [4]:
report_data

,Unnamed: 0,spectral_sample_id,barcode,crop_id,report_data
0,0,110990,AGR-FL-1684-38,38,"b'{""AGR-FL-1684-38"": {""results"": [{""result"": ""..."
1,1,110990,AGR-FL-1684-1,1,"b'{""AGR-FL-1684-1"": {""results"": [{""result"": ""6..."
2,2,110990,AGR-FL-1684-10,10,"b'{""AGR-FL-1684-10"": {""results"": [{""result"": ""..."
3,3,110994,AGR-FL-1671-38,38,"b'{""AGR-FL-1671-38"": {""results"": [{""result"": ""..."
4,4,110994,AGR-FL-1671-1,1,"b'{""AGR-FL-1671-1"": {""results"": [{""result"": ""6..."
...,...,...,...,...,...
202963,202963,248768,RWA-TS-3937-21,21,"b'{""RWA-TS-3937-21"": {""results"": [{""result"": ""..."
202964,202964,248771,RWA-TS-3936-21,21,"b'{""RWA-TS-3936-21"": {""results"": [{""result"": ""..."
202965,202965,206249,PAG-FL-1884-1,1,"b'{""PAG-FL-1884-1"": {""results"": [{""result"": ""5..."
202966,202966,149485,PAG-FL-0002-10,10,"b'{""PAG-FL-0002-10"": {""results"": [{""result"": ""..."


In [44]:
report_data.to_csv("inputFiles/report_data.csv")

In [18]:
comparison_df = pd.read_csv("inputFiles/approval-check.csv")    

In [5]:
reports = pd.DataFrame()


In [13]:
ct = 0
name = []
client = []
lat= []
lng = []
crop = []
for i in report_data['report_data']:
    ct+=1
    print(ct)
    # try:
    res = json.loads(eval(i))
    index = [j for j in res.keys()][0]
    print(index)
    farmer_dets = (res[index]['farmer_details'])
    name.append(farmer_dets['farmer_name'])
    client.append(farmer_dets['client_name'])
    lat.append(farmer_dets['lat'])
    lng.append(farmer_dets['lng'])
    crop.append(farmer_dets['crop'])
                
    continue
        
#         results = res[index]['results']
#         _df = pd.DataFrame(results)
#         _df['barcode'] = "-".join(index.split("-")[:-1])
#         _df['crop_code'] = index.split("-")[-1]
#         reports = pd.concat([reports, _df])
        
        
#     except Exception as e:
#         print(e)
# # reports.to_csv("inputFiles/reports_cleaned.csv")

1
AGR-FL-1684-38
2
AGR-FL-1684-1
3
AGR-FL-1684-10
4
AGR-FL-1671-38
5
AGR-FL-1671-1
6
AGR-FL-1671-10
7
AGR-FL-1501-38
8
AGR-FL-1501-1
9
AGR-FL-1501-10
10
AGR-FL-1503-38
11
AGR-FL-1503-1
12
AGR-FL-1503-10
13
AGR-FL-1504-38
14
AGR-FL-1504-1
15
AGR-FL-1504-10
16
AGR-FL-1505-38
17
AGR-FL-1505-1
18
AGR-FL-1505-10
19
AGR-FL-1506-38
20
AGR-FL-1506-1
21
AGR-FL-1506-10
22
AGR-FL-1507-38
23
AGR-FL-1507-1
24
AGR-FL-1507-10
25
AGR-FL-1509-38
26
AGR-FL-1509-1
27
AGR-FL-1509-10
28
AGR-FL-1511-38
29
AGR-FL-1511-1
30
AGR-FL-1511-10
31
AGR-FL-1523-38
32
AGR-FL-1523-1
33
AGR-FL-1523-10
34
AGR-FL-1517-38
35
AGR-FL-1517-1
36
AGR-FL-1517-10
37
AGR-FL-1528-38
38
AGR-FL-1528-1
39
AGR-FL-1528-10
40
AGR-FL-1514-38
41
AGR-FL-1514-1
42
AGR-FL-1514-10
43
AGR-FL-1520-38
44
AGR-FL-1520-1
45
AGR-FL-1520-10
46
AGR-FL-1532-38
47
AGR-FL-1532-1
48
AGR-FL-1532-10
49
AGR-FL-1535-38
50
AGR-FL-1535-1
51
AGR-FL-1535-10
52
AGR-FL-1536-38
53
AGR-FL-1536-1
54
AGR-FL-1536-10
55
AGR-FL-1537-38
56
AGR-FL-1537-1
57
AGR-FL-1537-10
58

TypeError: eval() arg 1 must be a string, bytes or code object

In [3]:
reports = pd.read_csv("inputFiles/reports_cleaned.csv")

C:\Users\tommy\AppData\Local\Temp\ipykernel_16880\4068765364.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  reports = pd.read_csv("inputFiles/reports_cleaned.csv")


In [4]:
reports_pivot = reports.copy(deep=True)

In [7]:
reports_pivot["barcode_crop"] = reports_pivot['barcode'] + "_" + reports_pivot['crop_code'].astype("str")

In [8]:
reports_pivot

,Unnamed: 0,result,status,result_unit,chemical_name,barcode,crop_code,0,barcode_crop
0,0,6.00,Low,%,ph,AGR-FL-1684,38,NaN,AGR-FL-1684_38
1,1,10-30,Low,ppm,available p,AGR-FL-1684,38,NaN,AGR-FL-1684_38
2,2,632,Very High,ppm,exchangeable k,AGR-FL-1684,38,NaN,AGR-FL-1684_38
3,3,1610,Optimum,ppm,calcium,AGR-FL-1684,38,NaN,AGR-FL-1684_38
4,4,294,Optimum,ppm,magnesium,AGR-FL-1684,38,NaN,AGR-FL-1684_38
...,...,...,...,...,...,...,...,...,...
2531601,13,15.5,Optimum,NaN,c/n ratio,OCP-PL-00140,180,NaN,OCP-PL-00140_180
2531602,14,66,High,%,clay,OCP-PL-00140,180,NaN,OCP-PL-00140_180
2531603,15,14.7,Low,%,silt,OCP-PL-00140,180,NaN,OCP-PL-00140_180
2531604,16,19.3,Low,%,sand,OCP-PL-00140,180,NaN,OCP-PL-00140_180


In [9]:
reports_pivot['chemical_name'] = reports_pivot['chemical_name'].str.lower().replace("/","").replace(" ","_").replace(".","")

In [10]:
reports_pivot['chemical_name'] = [str(i).replace("/","").replace(" ","_").replace(".","") for i in reports_pivot['chemical_name']]

In [11]:
reports_pivot = reports_pivot.reset_index()

In [12]:
reports_pivot = reports_pivot.dropna(subset="status")

In [13]:
reports_pivot['status'] = reports_pivot['status'].str.lower()

In [14]:
reports_pivot['status'] = [i.strip() for i in reports_pivot['status']]

In [15]:
reports_pivot.loc[reports_pivot['barcode'] == "1692083006981"]

,index,Unnamed: 0,result,status,result_unit,chemical_name,barcode,crop_code,0,barcode_crop
2360831,2360831,0,2.00,very low,NaN,ph,1692083006981,10,NaN,1692083006981_10
2360832,2360832,1,< 10,very low,ppm,available_p,1692083006981,10,NaN,1692083006981_10
2360833,2360833,2,< 40,very low,ppm,exchangeable_k,1692083006981,10,NaN,1692083006981_10
2360836,2360836,5,60,optimum,ppm,iron,1692083006981,10,NaN,1692083006981_10
2360837,2360837,6,60,optimum,ppm,manganese,1692083006981,10,NaN,1692083006981_10
2360838,2360838,7,60,very high,ppm,boron,1692083006981,10,NaN,1692083006981_10
2360839,2360839,8,60,very high,ppm,copper,1692083006981,10,NaN,1692083006981_10
2360840,2360840,9,60,very high,ppm,zinc,1692083006981,10,NaN,1692083006981_10
2360841,2360841,10,-37.3,very low,meq/100g,cec,1692083006981,10,NaN,1692083006981_10
2360842,2360842,11,3,very high,%,total_nitrogen,1692083006981,10,NaN,1692083006981_10


In [16]:
reports_pivot['status'].value_counts()

status
optimum      1108016
very low      408372
low           383478
very high     258147
high          111678
Name: count, dtype: int64

In [17]:
status_dict ={
    "very low":0,
    "low":1,
    "optimum":2,
    "high":3,
    "very high":3
}

status_dict_inv ={
    0: "very low",
    1: "low",
    2: "optimum",
    3: "high",
}

In [18]:
reports_pivot['status'] = [status_dict[i] for i in reports_pivot['status']]

In [19]:
reports_pivot.status.value_counts()

status
2    1108016
0     408372
1     383478
3     369825
Name: count, dtype: int64

In [20]:
reports_pivot_2 = pd.pivot_table(data=reports_pivot,index="barcode_crop",columns="chemical_name",values="status", aggfunc=max) 

C:\Users\tommy\AppData\Local\Temp\ipykernel_16880\902337865.py:1: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  reports_pivot_2 = pd.pivot_table(data=reports_pivot,index="barcode_crop",columns="chemical_name",values="status", aggfunc=max)


In [21]:
reports_pivot_2

chemical_name,available_p,boron,calcium,cec,clay,cn_ratio,copper,exchangeable_k,iron,magnesium,manganese,organic_matter,ph,sand,silt,total_nitrogen,zinc
barcode_crop,,,,,,,,,,,,,,,,,
0.P-GB_8669,0.0,1.0,2.0,0.0,1.0,NaN,1.0,2.0,2.0,2.0,2.0,0.0,2.0,3.0,1.0,0.0,2.0
0CP-AE_.3448,0.0,0.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,1.0,1.0
1692083006981_10,0.0,3.0,NaN,0.0,0.0,0.0,3.0,0.0,2.0,NaN,2.0,2.0,0.0,NaN,0.0,3.0,3.0
1693472359551_10,1.0,3.0,NaN,2.0,0.0,2.0,3.0,3.0,2.0,NaN,3.0,2.0,1.0,NaN,0.0,2.0,3.0
1693917106291_10,1.0,3.0,NaN,2.0,0.0,2.0,3.0,3.0,2.0,NaN,3.0,2.0,1.0,NaN,0.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
stk-fl-4293_48,0.0,0.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,1.0,2.0
stk-fl-4665_48,0.0,0.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,2.0,2.0
stk-fl-5145_48,0.0,0.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,1.0,2.0


In [22]:
reports_pivot_2 = reports_pivot_2.replace(3,status_dict_inv[3]).replace(2,status_dict_inv[2]).replace(1,status_dict_inv[1]).replace(0,status_dict_inv[0])

In [23]:
reports_pivot_2

chemical_name,available_p,boron,calcium,cec,clay,cn_ratio,copper,exchangeable_k,iron,magnesium,manganese,organic_matter,ph,sand,silt,total_nitrogen,zinc
barcode_crop,,,,,,,,,,,,,,,,,
0.P-GB_8669,very low,low,optimum,very low,low,NaN,low,optimum,optimum,optimum,optimum,very low,optimum,high,low,very low,optimum
0CP-AE_.3448,very low,very low,optimum,low,high,optimum,optimum,optimum,high,high,optimum,optimum,optimum,high,high,low,low
1692083006981_10,very low,high,NaN,very low,very low,very low,high,very low,optimum,NaN,optimum,optimum,very low,NaN,very low,high,high
1693472359551_10,low,high,NaN,optimum,very low,optimum,high,high,optimum,NaN,high,optimum,low,NaN,very low,optimum,high
1693917106291_10,low,high,NaN,optimum,very low,optimum,high,high,optimum,NaN,high,optimum,low,NaN,very low,optimum,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
stk-fl-4293_48,very low,very low,optimum,low,NaN,optimum,optimum,optimum,optimum,optimum,optimum,optimum,optimum,NaN,NaN,low,optimum
stk-fl-4665_48,very low,very low,optimum,optimum,NaN,optimum,optimum,optimum,optimum,optimum,optimum,optimum,optimum,NaN,NaN,optimum,optimum
stk-fl-5145_48,very low,very low,optimum,low,NaN,optimum,optimum,optimum,optimum,optimum,optimum,optimum,optimum,NaN,NaN,low,optimum


In [24]:
reports_pivot_2['crop_id'] = [str(i).split("_")[-1].strip() for i in reports_pivot_2.index]


In [25]:
reports_pivot_2['barcode'] = ["_".join(str(i).split("_")[:-1]).strip() for i in reports_pivot_2.index]

In [27]:
reports_pivot_2.to_csv("inputFiles/reports_final.csv")

In [20]:
reports_pivot_2 = pd.read_csv("inputFiles/reports_final.csv")

C:\Users\tommy\AppData\Local\Temp\ipykernel_14088\2138674722.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  reports_pivot_2 = pd.read_csv("inputFiles/reports_final.csv")


In [69]:
spectral_sample_df = pd.DataFrame()
conn = get_db_cursor()
cur = conn.cursor()
renaming = {}
for index, row in comparison_df.iterrows():
    os.makedirs("outputFiles/opus_renamed",exist_ok=True)
    os.makedirs("outputFiles/opus",exist_ok=True)
    barcode = row['barcode']
    if len([i for i in Path('./outputFiles/opus_renamed').rglob(f"*{barcode}*")]) == 2:
        continue
    print(barcode)
    crop = row['crop']
    ph = row['ph']
    phosphorus = row['phosphorus']
    potassium = row['potassium']
    organic_matter = row['Organic Matter']
    calcium = row['calcium']
    magnesium = row['magnesium']

    crop_id = pd.read_sql(f"SELECT id FROM crop c WHERE c.name = '{crop}'", con=conn).id.values[0]

    crop_reports = (reports_pivot_2.loc[reports_pivot_2['crop_id'] == str(crop_id)])
    if(ph in ['low','very low','optimum','high','very high']):
        crop_reports = crop_reports.loc[crop_reports['ph'] == ph]
    if(phosphorus in ['low','very low','optimum','high','very high']):
        crop_reports = crop_reports.loc[crop_reports['available_p'] == phosphorus]
    if(potassium in ['low','very low','optimum','high','very high']):
        crop_reports = crop_reports.loc[crop_reports['exchangeable_k'] == potassium]
    if(calcium in ['low','very low','optimum','high','very high']):
        crop_reports = crop_reports.loc[crop_reports['calcium'] == calcium]
    if(magnesium in ['low','very low','optimum','high','very high']):
        crop_reports = crop_reports.loc[crop_reports['magnesium'] == magnesium]
    if(len(crop_reports) == 0):
        continue
    spectral_df = pd.read_sql(f"SELECT spectral_sample_id, barcode, spectral_batch_id, latitude, longitude, farmer_name, sampler_name, previous_crop, next_crop, sample_date, phone_number, analysis_type, tree_population,  TIMESTAMPDIFF(YEAR,'2024-17-05',date_of_planting) as 'age', field_size  FROM SpectralSample WHERE barcode IN {tuple(crop_reports['barcode'].values)}", con=conn)
    spectral_df = spectral_df.loc[spectral_df['spectral_batch_id'] > 3355]
    
    folder = f"./outputFiles/{crop}_ph-{ph}_phosphorus-{phosphorus}_potassium-{potassium}_organicmatter-{organic_matter}_calcium-{calcium}_magnesium-{magnesium}"
    os.makedirs(folder, exist_ok=True)
    print(f"======================{folder}======================")
    for index, row in spectral_df.iterrows():
        batch_no = row['spectral_batch_id']
        print(f"Downloading batch no {batch_no}")
        if len([i for i in Path('./outputFiles/opus_renamed').rglob(f"*{barcode}*")]) == 2:
            break
        try:
            subprocess.run(f"scp -r root@161.35.160.152:/mnt/volume_lon1_01/spc_backup/batch_{batch_no} {folder}")
            
            count = 0
            for file in [i for i in Path(folder).rglob("**/*.0")][::-1]:
                if(count==2):
                    break
                directory = file.parent
                name = file.name
                actual_barcode = name.split("_")[0]
                
                shutil.copyfile(file,f"outputFiles/opus/{name}")
                os.rename(file,f"outputFiles/opus_renamed/{barcode}_{count}.0")
                renaming[actual_barcode] = barcode
                row['barcode'] = barcode
                row['Report Language'] = 'en'
                _ = pd.DataFrame(row).T
                print(spectral_sample_df)
                spectral_sample_df = pd.concat([spectral_sample_df, _])
                print(spectral_sample_df)
                count+=1
            
        except Exception as e:
            print(e)
subprocess.run(f"powershell Compress-Archive {os.getcwd()}\\outputFiles\\opus_renamed {os.getcwd()}\\outputFiles\\opus.zip")
spectral_sample_df = spectral_sample_df.drop_duplicates()
spectral_sample_df.to_csv("outputFiles/Spectral Sample Output.csv")   

TEST-DS1-0001


C:\Users\tommy\AppData\Local\Temp\ipykernel_14088\1937457372.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  crop_id = pd.read_sql(f"SELECT id FROM crop c WHERE c.name = '{crop}'", con=conn).id.values[0]
C:\Users\tommy\AppData\Local\Temp\ipykernel_14088\1937457372.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spectral_df = pd.read_sql(f"SELECT spectral_sample_id, barcode, spectral_batch_id, latitude, longitude, farmer_name, sampler_name, previous_crop, next_crop, sample_date, phone_number, analysis_type, tree_population,  TIMESTAMPDIFF(YEAR,'2024-17-05',date_of_planting) as 'age', field_size  FROM SpectralSample WHERE barcode IN {tuple(crop_reports['barcode'].values

======================./outputFiles/Maize_ph-high_phosphorus-high_potassium-high_organicmatter-nan_calcium-nan_magnesium-nan======================
Empty DataFrame
Columns: []
Index: []
  spectral_sample_id        barcode spectral_batch_id  latitude  longitude  \
8             138593  TEST-DS1-0001              3970 -1.126175  36.641971   

                                         farmer_name   sampler_name  \
8  Cropnuts Spectral Test (Digifarm Calibration  ...  Martin Wafula   

  previous_crop next_crop          sample_date phone_number  \
8         Maize     Maize  2021-09-28 07:55:52    720759475   

            analysis_type tree_population   age field_size Report Language  
8  Starter Soil Scan (IR)            None  None        1.0              en  
  spectral_sample_id        barcode spectral_batch_id  latitude  longitude  \
8             138593  TEST-DS1-0001              3970 -1.126175  36.641971   

                                         farmer_name   sampler_name  \
8  Cro

C:\Users\tommy\AppData\Local\Temp\ipykernel_14088\1937457372.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  crop_id = pd.read_sql(f"SELECT id FROM crop c WHERE c.name = '{crop}'", con=conn).id.values[0]
C:\Users\tommy\AppData\Local\Temp\ipykernel_14088\1937457372.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spectral_df = pd.read_sql(f"SELECT spectral_sample_id, barcode, spectral_batch_id, latitude, longitude, farmer_name, sampler_name, previous_crop, next_crop, sample_date, phone_number, analysis_type, tree_population,  TIMESTAMPDIFF(YEAR,'2024-17-05',date_of_planting) as 'age', field_size  FROM SpectralSample WHERE barcode IN {tuple(crop_reports['barcode'].values

======================./outputFiles/Maize_ph-low_phosphorus-high_potassium-low_organicmatter-optimum_calcium-nan_magnesium-nan======================
  spectral_sample_id        barcode spectral_batch_id  latitude  longitude  \
8             138593  TEST-DS1-0001              3970 -1.126175  36.641971   
8             138593  TEST-DS1-0001              3970 -1.126175  36.641971   

                                         farmer_name   sampler_name  \
8  Cropnuts Spectral Test (Digifarm Calibration  ...  Martin Wafula   
8  Cropnuts Spectral Test (Digifarm Calibration  ...  Martin Wafula   

  previous_crop next_crop          sample_date phone_number  \
8         Maize     Maize  2021-09-28 07:55:52    720759475   
8         Maize     Maize  2021-09-28 07:55:52    720759475   

            analysis_type tree_population   age field_size Report Language  
8  Starter Soil Scan (IR)            None  None        1.0              en  
8  Starter Soil Scan (IR)            None  None        1.0

...

======================./outputFiles/Beans_ph-high_phosphorus-high_potassium-high_organicmatter-low_calcium-nan_magnesium-nan======================
   spectral_sample_id        barcode spectral_batch_id   latitude  longitude  \
8              138593  TEST-DS1-0001              3970  -1.126175  36.641971   
8              138593  TEST-DS1-0001              3970  -1.126175  36.641971   
21             192981  TEST-DS1-0002              6338 -13.975737  33.286332   
21             192981  TEST-DS1-0002              6338 -13.975737  33.286332   
10             114303  TEST-DS1-0003              3384  -7.614637  31.288057   
10             114303  TEST-DS1-0003              3384  -7.614637  31.288057   

                                          farmer_name     sampler_name  \
8   Cropnuts Spectral Test (Digifarm Calibration  ...    Martin Wafula   
8   Cropnuts Spectral Test (Digifarm Calibration  ...    Martin Wafula   
21                              Brandina Kalanza(LOW)  Malumbo Kapunda

C:\Users\tommy\AppData\Local\Temp\ipykernel_14088\1937457372.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  crop_id = pd.read_sql(f"SELECT id FROM crop c WHERE c.name = '{crop}'", con=conn).id.values[0]
C:\Users\tommy\AppData\Local\Temp\ipykernel_14088\1937457372.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spectral_df = pd.read_sql(f"SELECT spectral_sample_id, barcode, spectral_batch_id, latitude, longitude, farmer_name, sampler_name, previous_crop, next_crop, sample_date, phone_number, analysis_type, tree_population,  TIMESTAMPDIFF(YEAR,'2024-17-05',date_of_planting) as 'age', field_size  FROM SpectralSample WHERE barcode IN {tuple(crop_reports['barcode'].values

======================./outputFiles/Beans_ph-low_phosphorus-low_potassium-low_organicmatter-low_calcium-nan_magnesium-nan======================
   spectral_sample_id        barcode spectral_batch_id   latitude  longitude  \
8              138593  TEST-DS1-0001              3970  -1.126175  36.641971   
8              138593  TEST-DS1-0001              3970  -1.126175  36.641971   
21             192981  TEST-DS1-0002              6338 -13.975737  33.286332   
21             192981  TEST-DS1-0002              6338 -13.975737  33.286332   
10             114303  TEST-DS1-0003              3384  -7.614637  31.288057   
10             114303  TEST-DS1-0003              3384  -7.614637  31.288057   
0              171241  TEST-DS1-0004              5602   -6.79734   32.02026   
0              171241  TEST-DS1-0004              5602   -6.79734   32.02026   

                                          farmer_name     sampler_name  \
8   Cropnuts Spectral Test (Digifarm Calibration  ...    Mart

C:\Users\tommy\AppData\Local\Temp\ipykernel_14088\1937457372.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  crop_id = pd.read_sql(f"SELECT id FROM crop c WHERE c.name = '{crop}'", con=conn).id.values[0]
C:\Users\tommy\AppData\Local\Temp\ipykernel_14088\1937457372.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spectral_df = pd.read_sql(f"SELECT spectral_sample_id, barcode, spectral_batch_id, latitude, longitude, farmer_name, sampler_name, previous_crop, next_crop, sample_date, phone_number, analysis_type, tree_population,  TIMESTAMPDIFF(YEAR,'2024-17-05',date_of_planting) as 'age', field_size  FROM SpectralSample WHERE barcode IN {tuple(crop_reports['barcode'].values

======================./outputFiles/Maize_ph-low_phosphorus-nan_potassium-nan_organicmatter-nan_calcium-low_magnesium-optimum======================
   spectral_sample_id        barcode spectral_batch_id   latitude  longitude  \
8              138593  TEST-DS1-0001              3970  -1.126175  36.641971   
8              138593  TEST-DS1-0001              3970  -1.126175  36.641971   
21             192981  TEST-DS1-0002              6338 -13.975737  33.286332   
21             192981  TEST-DS1-0002              6338 -13.975737  33.286332   
10             114303  TEST-DS1-0003              3384  -7.614637  31.288057   
10             114303  TEST-DS1-0003              3384  -7.614637  31.288057   
0              171241  TEST-DS1-0004              5602   -6.79734   32.02026   
0              171241  TEST-DS1-0004              5602   -6.79734   32.02026   
0              175953  TEST-DS1-0006              5557   0.335837  34.531549   
0              175953  TEST-DS1-0006              55

In [70]:
spectral_sample_df

,spectral_sample_id,barcode,spectral_batch_id,latitude,longitude,farmer_name,sampler_name,previous_crop,next_crop,sample_date,phone_number,analysis_type,tree_population,age,field_size,Report Language
8,138593,TEST-DS1-0001,3970,-1.126175,36.641971,Cropnuts Spectral Test (Digifarm Calibration ...,Martin Wafula,Maize,Maize,2021-09-28 07:55:52,720759475,Starter Soil Scan (IR),None,None,1.0,en
21,192981,TEST-DS1-0002,6338,-13.975737,33.286332,Brandina Kalanza(LOW),Malumbo Kapunda,Maize,Maize,2022-11-16 12:59:21,991833404,Soil Test Report,None,None,1.0,en
10,114303,TEST-DS1-0003,3384,-7.614637,31.288057,Adolf Myula,Samuel Juma,Maize,Maize,2021-05-25 12:09:43,0628794578,Soil Test Report,None,None,1.0,en
0,171241,TEST-DS1-0004,5602,-6.79734,32.02026,FABIANO FUTAKAMBA NKUNDA,Samiu Yussuph,Maize,Beans,2022-07-12 12:23:11,0627498477,Soil Test Report,None,None,1.0,en
0,175953,TEST-DS1-0006,5557,0.335837,34.531549,Alex Livgstone Ongwen (Alex Farm - Field Two),Norbbert Okutu,Maize,Beans,2022-07-30 12:18:13,720230509,Starter Soil Scan (IR),None,None,1.0,en
304,113128,TEST-DS1-0007,3357,-0.098459,35.562261,Francis Mwangi,Jack Omondi Ongala,Pyrethrum,Pyrethrum,2021-03-16 10:24:46,710704920,Starter Soil Scan (IR),None,None,0.0,en
